##  Introduction of Support Vector Machine

支持向量机（Support Vector Machine, SVM）是一类按监督学习（supervised learning）方式对数据进行二元分类的广义线性分类器（generalized linear classifier），其决策边界是对学习样本求解的最大边距超平面(maximum-margin hyperplane)。支持向量机(svm)的想法与前面介绍的感知机模型类似，找一个超平面将正负样本分开，但svm的想法要更深入了一步，它要求正负样本中离超平面最近的点的距离要尽可能的大，所以svm模型建模可以分为两个子问题： 
1. 将正负样本尽可能分开
2. 让两类样本离超平面距离尽可能远

## Definition of SVM

SVM的目标函数可以表示为：
$$
f(x)=sign(w^Tx+b)
$$  

对于样本对$(x,y)$，只要$w^Tx+b$和$y$的符号相同，就意味着分对了，即$y\cdot (w^Tx+b)>0$。解决了分对的问题后要解决怎样使得超平面离两类样本点尽可能远即
$$
\max ~d
$$
当达到要求时，超平面距离正负样本的距离一定是一样的，点到超平面的距离表示为：
$$
d=\frac{|w^Tx+b|}{||w||}
$$  
参数$w,b$可以等比例的变化，而不会影响到模型自身，所以$|w^Tx+b|=1$自然也可以满足，所以这时最近的点的距离可以表示为：

$$
d^*=\frac{1}{||w||}
$$
同时第一个子问题的条件要调整为$y\cdot(w^Tx+b)\geq1$，而$\max d^*$可以等价的表示为$\min \frac{1}{2}w^Tw$，所以svm模型的求解可以表述为如下优化问题：  

$$
\max _{w,b} \frac{1}{w^Tw} \\
s.t.~~~~y_i(w^Tx_i+b)\geq 1,i=1,2,...,N
$$
通常情况下我们习惯写成最小化问题，因此可以重新表示为：
$$
\min_{w,b} \frac{1}{2}w^Tw \\
s.t.~~~~y_i(w^Tx_i+b)\geq 1,i=1,2,...,N
$$

## Lagrangian Duality

对于上面优化问题的求解往往转化为对其对偶问题的求解，首先，构造其拉格朗日函数：
$$
L(w,b,\lambda)=\frac{1}{2}w^Tw+\sum_{i=1}^N \lambda_i(1-y_i(w^Tx_i+b)),\lambda=[\lambda_1,\lambda_2,...,\lambda_N]
$$  
这时，原优化问题（设为$P$）就等价于：  

$$
\min_{w,b}\max_{\lambda}L(w,b,\lambda)\\
s.t.~~~~\lambda_i\geq 0,i=1,2,...,N
$$  



### Karush–Kuhn–Tucker conditions

首先最优解一定满足原问题的约束条件：
$$
1-y_i({w^*}^Tx_i+b)\leq0,i=1,2,...,N
$$  
其次还要满足对偶问题的约束条件：
$$
\lambda_i^*\geq0,i=1,2,...,N\\
$$
互补松弛条件：
$$
\lambda_i^*(1-y_i({w^*}^Tx_i+b^*))=0,i=1,2,...,N
$$  
$L(w,b,\lambda)$关于$w,b$的偏导为0，即：
$$
w=\sum_{i=1}^N\lambda_iy_ix_i\\
0=\sum_{i=1}^N\lambda_iy_i
$$
将上述两式$w$和$b$带入对偶问题可得：
$$
\max_{\lambda} \sum_{i=1}^N\lambda_i-\frac{1}{2}\sum_{i=1}^N\sum_{j=1}^N\lambda_i\lambda_jy_iy_jx_i^Tx_j\\
s.t.\sum_{i=1}^N\lambda_iy_i=0,\\
\lambda\geq0,i=1,2,...,N
$$  
将最大化问题转化为最小化问题有：
$$
\min_{\lambda} \frac{1}{2}\sum_{i=1}^N\sum_{j=1}^N\lambda_i\alpha_jy_iy_jx_i^Tx_j-\sum_{i=1}^N\lambda_i\\
s.t.\sum_{i=1}^N\lambda_iy_i=0,\\
\lambda_i\geq0,i=1,2,...,N
$$
支持向量机问题在模型上非常简单，但是其求解却相当困难。SVM的对偶问题使用二次规划的软件包一定可以暴力求解得到最优解，但是它的复杂度非常高，主要原因有：
1. 变量数与样本数相同，每个变量$\lambda_i$对应样本$(x_i,y_i)$；
2. 约束条件数也与样本数相同；


这里我们假设$\lambda_i$已经求出来了，那么根据$w=\sum_{i=1}^N\lambda_iy_ix_i$可以求出最优的$w^*$，对于b可以通过以下公式求得：
$$
b = -\frac{\max_{i,y_i=-1}w^{*T}x_i+\min_{i,y_i=1}w^{*T}x_i}{2}
$$

可以看出一旦$\lambda_i$求出，那么$w$和$b$也可以相应确定，因此问题的关键是求解$\lambda_i$

## SMO algorithm for solving SVM

序列最小优化算法(Sequential minimal optimization, SMO)是一种用于解决支持向量机训练过程中所产生优化问题的算法。SMO由微软研究院的约翰·普莱特于1998年发明，被广泛使用于SVM的训练过程中，并在通行的SVM库LIBSVM中得到实现。1998年，SMO算法发表在SVM研究领域内引起了轰动，因为先前可用的SVM训练方法必须使用复杂的方法，并需要昂贵的第三方二次规划工具。而SMO算法较好地避免了这一问题。 

序列最小最优化化(sequential minimal optimization,SMO)算法是求解SVM对偶问题的一种启发式算法，它的思路是：**每次只选择一个变量优化，而固定住其他变量**。

比如选择$\lambda_1$进行优化，而固定住$\lambda_i,i=2,3,...,N$，但由于我们的问题中有一个约束：$\sum_i^N\lambda_iy_i=0$，需要另外选择一个$\lambda_2$来配合$\lambda_1$做改变，当两者中任何一个变量确定后，另外一个也就随之确定了，比如确定$\lambda_2$后：  

$$
\lambda_1=-y_i\sum_{i=2}^N\lambda_iy_i
$$  